In [1]:
DOMAIN = 'test' # test, spouse, cdr

In [2]:
import sys
import os

if DOMAIN == 'cdr':
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 65


In [4]:
from snorkel.semantic import (get_left_tokens, get_right_tokens, 
                              get_between_tokens, get_sentence_tokens)
c = candidates[0]
# print c[0].get_parent()._asdict()['ner_tags']
# print get_left_tokens(c[0])
# print [getattr(t, 'words') for t in get_left_tokens(c[0])]

In [5]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [6]:
# c = sv.get_selected()
# print c

In [7]:
# hash(c)

In [8]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'cdr':
    user_lists = {}

In [9]:
# examples[0].display()

In [10]:
# examples[28].candidate[0].get_parent()

In [11]:
%time from snorkel.semantic import SemanticParser

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 18.1 µs


In [12]:
%time sp = SemanticParser(candidate_class, user_lists, absorb=True)

Created grammar with 257 rules
CPU times: user 5.27 ms, sys: 11.1 ms, total: 16.4 ms
Wall time: 59.9 ms


In [13]:
%time results = sp.evaluate(examples, show_explanation=True, show_correct=True, only=[35])
results

Example 35: label True because there are not three people in the sentence
C: ('.root', ('.label', ('.bool', True), ('.call', ('.not_equals', ('.int', 3)), ('.count', ('.filter_attr', ('.sentence_tokens',), ('.string', 'ner_tags'), ('.string', 'PERSON'))))))
($ROOT ($LF ($Label label) ($LF_$Label ($Bool ($BoolLit ($True true))) ($LF_$Label_$Bool ($Because ($Because because) ($QueryToken there)) ($Bool ($IntToBool ($Compare ($NotEquals ($Equals ($Is are)) ($Not not))) ($Int three)) ($List ($TokenList ($NER ($PersonNER people)) ($TokenList ($Sentence in the sentence)))))))))
C: ('.root', ('.label', ('.bool', True), ('.call', ('.not_equals', ('.int', 3)), ('.count', ('.field', ('.filter_attr', ('.sentence_tokens',), ('.string', 'ner_tags'), ('.string', 'PERSON')), ('.string', 'words'))))))
($ROOT ($LF ($Label label) ($LF_$Label ($Bool ($BoolLit ($True true))) ($LF_$Label_$Bool ($Because ($Because because) ($QueryToken there)) ($Bool ($IntToBool ($Compare ($NotEquals ($Equals ($Is are)) ($N

,Correct,Incorrect,Redundant,Failed,Unknown
Example35,8,6,154,0,0


In [14]:
import numpy as np
print np.sum(results, axis=0)

Correct        8
Incorrect      6
Redundant    154
Failed         0
Unknown        0
dtype: int64


In [17]:
sp.grammar.print_chart(nested=True)

(0, 1)      
             ($QueryToken label)
             ($Label label)
(1, 2)      
             ($QueryToken true)
             ($True true)
             ($BoolLit ($True true))
             ($Bool ($BoolLit ($True true)))
(2, 3)      
             ($QueryToken because)
             ($Because because)
(3, 4)      
             ($QueryToken there)
(4, 5)      
             ($QueryToken are)
             ($Is are)
             ($Exists ($Is are))
             ($Equals ($Is are))
             ($Compare ($Equals ($Is are)))
             ($BinaryStringToBool ($Compare ($Equals ($Is are))))
(5, 6)      
             ($QueryToken not)
             ($Not not)
(6, 7)      
             ($QueryToken three)
             ($Int three)
(7, 8)      
             ($QueryToken people)
             ($PersonNER people)
             ($NER ($PersonNER people))
(8, 9)      
             ($QueryToken in)
             ($In in)
             ($BinaryStringToBool ($In in))
(9, 10)     
             ($QueryTo

In [16]:
# sp.grammar.print_grammar()